<a href="https://colab.research.google.com/github/towardsai/ai-tutor-rag-system/blob/main/notebooks/04-RAG_with_VectorStore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Packages and Setup Variables

In [1]:
!pip install -q llama-index==0.10.5 llama-index-vector-stores-chroma==0.1.7 langchain==0.1.17 langchain-chroma==0.1.0 langchain_openai==0.1.5 openai==1.12.0 cohere==4.47 tiktoken==0.6.0 chromadb==0.4.22

In [2]:
import os

# Set the "OPENAI_API_KEY" in the Python environment. Will be used by OpenAI client later.
os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_KEY>"

# Load the Dataset (CSV)

## Download

The dataset includes several articles from the TowardsAI blog, which provide an in-depth explanation of the LLaMA2 model. Read the dataset as a long string.

In [3]:
!curl -o ./mini-dataset.csv https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  169k  100  169k    0     0   277k      0 --:--:-- --:--:-- --:--:--  281k


## Read File

In [4]:
import csv

text = ""

# Load the file as a JSON
with open("./mini-dataset.csv", mode="r", encoding="ISO-8859-1") as file:
  csv_reader = csv.reader(file)

  for row in csv_reader:
    text += row[0]

# The number of characters in the dataset.
len( text )

841

# Chunking

In [5]:
chunk_size = 512
chunks = []

# Split the long text into smaller manageable chunks of 512 characters.
for i in range(0, len(text), chunk_size):
    chunks.append(text[i:i + chunk_size])

len( chunks )

2

#Interface of Chroma with LlamaIndex

In [6]:
from llama_index.core import Document

# Convert the chunks to Document objects so the LlamaIndex framework can process them.
documents = [Document(text=t) for t in chunks]

Save on Chroma



In [7]:
import chromadb

# create client and a new collection
# chromadb.EphemeralClient saves data in-memory.
chroma_client = chromadb.PersistentClient(path="./mini-chunked-dataset")
chroma_collection = chroma_client.create_collection("mini-chunked-dataset")

In [8]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
# Define a storage context object using the created vector database.
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [9]:
from llama_index.core import VectorStoreIndex

# Add the documents to the database and create Index / embeddings
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

Query Dataset

In [10]:
from llama_index.llms.openai import OpenAI
# Define a query engine that is responsible for retrieving related pieces of text,
# and using a LLM to formulate the final answer.

llm = OpenAI(temperature=0, model="gpt-3.5-turbo", max_tokens=512)
query_engine = index.as_query_engine(llm=llm)

In [11]:
response = query_engine.query(
    "How many parameters LLaMA2 model has?"
)
print(response)

The LLaMA2 model has 7 billion parameters.


#Interface of Chroma with LangChain

In [12]:
from langchain.schema.document import Document
# Convert the chunks to Document objects so the LangChain framework can process them.
documents = [Document(page_content=t) for t in chunks]

Save on Chroma

In [13]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
# Add the documents to chroma DB and create Index / embeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
chroma_db = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory="./mini-chunked-dataset",
    collection_name="mini-chunked-dataset"
)

Query Dataset

In [14]:
from langchain_openai import ChatOpenAI
# Initializing the LLM model
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", max_tokens=512)

In [16]:
from langchain.chains import RetrievalQA
query = "How many parameters LLaMA2 model has?"
retriever = chroma_db.as_retriever(search_kwargs={"k": 2})
# Define a RetrievalQA chain that is responsible for retrieving related pieces of text,
# and using a LLM to formulate the final answer.
chain = RetrievalQA.from_chain_type(llm=llm,
                                    chain_type="stuff",
                                    retriever=retriever)

response = chain(query)
print(response["result"])

The LLaMA-2 model has 7 billion parameters.
